In [7]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('/content/drive/My Drive/codes/digits.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Now we split the image to 5000 cells, each 20x20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Make it into a Numpy array. It size will be (50,100,20,20)
x = np.array(cells)

# Now we prepare train_data and test_data.
x_train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
x_test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# Create labels for train and test data
#k = np.arange(10)
#train_labels = np.repeat(k,250)[:,np.newaxis]
#test_labels = train_labels.copy()
print(x_train.shape)
print(x_test.shape)
Y = np.array([[1,0,0,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,0,0],
              [0,0,0,0,1,0,0,0,0,0],
              [0,0,0,0,0,1,0,0,0,0],
              [0,0,0,0,0,0,1,0,0,0],
              [0,0,0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,0,0,0,1]])

Y = np.repeat(Y, 250, axis = 0)
#print(Y.shape)
y_train = Y
print(y_train.shape)

testimg = cv2.imread('/content/drive/My Drive/codes/img_114.jpg',0)
#testimg = np.invert(testimg)
testimg = cv2.resize(testimg,(20, 20), interpolation = cv2.INTER_CUBIC)
#testimg = cv2.resize(testimg,(20, 20), interpolation = cv2.INTER_LINEAR)
#testimg = cv2.resize(testimg,(20, 20), interpolation = cv2.INTER_CUBIC)
#cv2.imshow('testimg',testimg)
testimg = testimg.reshape(-1,400).astype(np.float32)

def sigmoid(s):
    return 1/(1 + np.exp(-s))

def sigmoid_derv(s):
    return s * (1 - s)

def softmax(s):
    exps = np.exp(s - np.max(s, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1, keepdims=True)

def cross_entropy(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def error(pred, real):
    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss

class MyNN:
    def __init__(self, x, y):
        self.x = x
        neurons = 400
        self.lr = 0.5
        ip_dim = x.shape[1]
        op_dim = y.shape[1]

        self.w1 = np.random.randn(ip_dim, neurons)
        self.b1 = np.zeros((1, neurons))
        self.w2 = np.random.randn(neurons, neurons)
        self.b2 = np.zeros((1, neurons))
        self.w3 = np.random.randn(neurons, op_dim)
        self.b3 = np.zeros((1, op_dim))
        self.y = y

    def feedforward(self):
        z1 = np.dot(self.x, self.w1) + self.b1
        self.a1 = sigmoid(z1)
        z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(z2)
        z3 = np.dot(self.a2, self.w3) + self.b3
        self.a3 = softmax(z3)
        
    def backprop(self):
        loss = error(self.a3, self.y)
        print('Error :', loss)
        a3_delta = cross_entropy(self.a3, self.y) # w3
        z2_delta = np.dot(a3_delta, self.w3.T)
        a2_delta = z2_delta * sigmoid_derv(self.a2) # w2
        z1_delta = np.dot(a2_delta, self.w2.T)
        a1_delta = z1_delta * sigmoid_derv(self.a1) # w1

        self.w3 -= self.lr * np.dot(self.a2.T, a3_delta)
        self.b3 -= self.lr * np.sum(a3_delta, axis=0, keepdims=True)
        self.w2 -= self.lr * np.dot(self.a1.T, a2_delta)
        self.b2 -= self.lr * np.sum(a2_delta, axis=0)
        self.w1 -= self.lr * np.dot(self.x.T, a1_delta)
        self.b1 -= self.lr * np.sum(a1_delta, axis=0)

    def predict(self, data):
        self.x = data
        self.feedforward()
        return self.a3.argmax()
			
model = MyNN(x_train, y_train)

epochs = 1500
for x in range(epochs):
    model.feedforward()
    model.backprop()
		
def get_acc(x, y):
    acc = 0
    for xx,yy in zip(x, y):
        s = model.predict(xx)
        if s == np.argmax(yy):
            acc +=1
    return acc/len(x)*100
	
print("Training accuracy : ", get_acc(x_train, y_train))
print("Test accuracy : ", get_acc(x_test, y_train))
#print("Prediction : ", get_acc(testimg, ))
    
    
    



(2500, 400)
(2500, 400)
(2500, 10)
Error : 21.22157402003567


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: RuntimeWarning: overflow encountered in exp


Error : 17.267735430195998
Error : 13.663044601897132
Error : 11.526185087843267
Error : 10.114881510843531
Error : 8.705931976424253
Error : 7.664626396934248
Error : 7.027064844155926
Error : 6.103433938189617
Error : 5.948183780277536
Error : 4.356899466405668
Error : 3.9254531582041263
Error : 3.1760310657875395
Error : 3.295118604422618
Error : 3.6778163489967812
Error : 3.3087172448654645
Error : 3.5530097088671218
Error : 3.143548751509097
Error : 3.498030548450594
Error : 2.4485841427633392
Error : 2.702277512853818
Error : 2.6126401866524187
Error : 2.5756003336051703
Error : 2.2600255175603894
Error : 2.464357277823319
Error : 2.2021384811064855
Error : 2.1941671195202734
Error : 1.989776236425366
Error : 1.6603383640636458
Error : 1.6926689282664702
Error : 1.6768508031176985
Error : 1.6800467914187516
Error : 1.6123284371936233
Error : 1.7051143461374314
Error : 1.536314251544233
Error : 1.5589825884471076
Error : 1.401367244140884
Error : 1.4357948784076109
Error : 1.28143